In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import doctest

In [3]:
import abc
import random


class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        pass

    @abc.abstractmethod
    def pick(self):
        pass

    def loaded(self):
        return bool(self.inspect())

    def inspect(self):
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break

        self.load(items)
        return tuple(sorted(items))


class BingoCage(Tombola):

    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)

    def pick(self):
        cls = type(self)
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError(f'pick from empty {cls.__name__}')

    def __call__(self):
        return self.pick()

In [4]:
class AddableBingoCage(BingoCage):

    def __add__(self, other):
        cls = type(self)
        if isinstance(other, Tombola):
            return cls(self.inspect() + other.inspect())

        return NotImplemented

    def __iadd__(self, other):
        if isinstance(other, Tombola):
            other_iterable = other.inspect()
        else:
            try:
                other_iterable = iter(other)
            except TypeError:
                msg = 'right operand in += must be {!r} or an iterable'
                raise TypeError(msg.format(cls.__name__))

        self.load(other_iterable)
        return self


"""

>>> bc = AddableBingoCage([1, 2, 3])
>>> bc2 = AddableBingoCage([4, 5, 6])
>>> (bc + bc2).inspect()
(1, 2, 3, 4, 5, 6)
>>> bc += bc2
>>> bc.inspect()
(1, 2, 3, 4, 5, 6)
"""

doctest.testmod()

TestResults(failed=0, attempted=5)